In [9]:
from __future__ import annotations

import glob
import json
from pathlib import Path
from typing import Iterable, List, Optional

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

pd.set_option("display.max_colwidth", 200)



In [10]:
# Configuration: glob all exps_performance runs (JSONL)
ROOT = Path("/nlpgpu/data/terry/ToolProj/src/exps_performance/results")
MODEL_INCLUDE: list[str] = []  # e.g., ["gpt-oss", "gpt-5"] to filter; empty means all

run_paths = sorted(ROOT.glob("**/tb/run_*/res.jsonl"))
if MODEL_INCLUDE:
    run_paths = [p for p in run_paths if any(tok in p.as_posix() for tok in MODEL_INCLUDE)]
RUN_PATHS = [p for p in run_paths if p.exists()]
if not RUN_PATHS:
    raise FileNotFoundError("No res.jsonl found. Check ROOT/MODEL_INCLUDE.")

# Optional cap for quick iteration; set to None to load everything.
MAX_ROWS: Optional[int] = None



In [11]:
def load_jsonl(paths: Iterable[Path], limit: Optional[int] = None) -> pd.DataFrame:
    """Load GPTOSS result jsonl files into a DataFrame.

    Adds run metadata and keeps malformed lines (if any) flagged for visibility.
    """
    records: List[dict] = []
    for path in paths:
        seen = 0
        with path.open() as f:
            for lineno, line in enumerate(f, start=1):
                if not line.strip():
                    continue
                if limit is not None and seen >= limit:
                    break
                seen += 1
                try:
                    obj = json.loads(line)
                    obj.update({
                        "run_path": str(path),
                        "run_id": path.parent.name,
                        "run_model": obj.get("model", Path(path).parent.parent.parent.name),
                        "lineno": lineno,
                    })
                    records.append(obj)
                except json.JSONDecodeError as exc:  # keep the row so we can surface the failure
                    records.append({
                        "run_path": str(path),
                        "run_id": path.parent.name,
                        "run_model": Path(path).parent.parent.parent.name,
                        "lineno": lineno,
                        "load_parse_err": True,
                        "load_err": str(exc),
                        "raw": line,
                    })
    return pd.DataFrame.from_records(records)


def strip_fences(text: Optional[str]) -> str:
    if not isinstance(text, str):
        return ""
    cleaned = text.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.split("\n", 1)[1] if "\n" in cleaned else ""
    if cleaned.endswith("```"):
        cleaned = cleaned.rsplit("```", 1)[0].strip()
    return cleaned


def maybe_json_like(text: str) -> bool:
    return isinstance(text, str) and text.strip()[:1] in {"{", "["}


def safe_json_parse(text: str) -> tuple[Optional[dict], bool]:
    if not maybe_json_like(text):
        return None, False
    try:
        return json.loads(text), False
    except Exception:
        return None, True


def task_type(row: pd.Series) -> str:
    if row.get("code_question"):
        return "code"
    if row.get("nl_question"):
        return "nl"
    if row.get("sim_question"):
        return "sim"
    return "unknown"


def normalize_ok(text: Optional[str]) -> str:
    if text is None:
        return ""
    t = str(text).strip()
    return "" if t.lower() in {"ok", "ok,ok"} else t


def classify_issue(row: pd.Series) -> str:
    if row.get("load_parse_err"):
        return "load_parse_err"
    if row.get("parse_err_flag"):
        return "parse_error"
    if row.get("code_err_flag"):
        return "code_error"
    if row.get("missing_output"):
        return "missing_output"
    if not row.get("is_correct", False):
        return "wrong_answer"
    return "ok"


def derive_flags(df: pd.DataFrame) -> pd.DataFrame:
    answer_field = {
        "sim": "sim_answer",
        "code": "code_answer",
        "nl": "nl_answer",
    }

    def pick(row, suffix, default=False):
        key = f"{row['task_type']}_{suffix}"
        return row.get(key, default)

    df = df.copy()
    df["task_type"] = df.apply(task_type, axis=1)
    df["raw_answer"] = df.apply(lambda r: r.get(answer_field.get(r["task_type"], "sim"), ""), axis=1)
    df["clean_answer"] = df["raw_answer"].apply(strip_fences)
    df["json_parse_err"] = df["clean_answer"].apply(lambda t: safe_json_parse(t)[1])
    df["missing_output"] = df["clean_answer"].apply(lambda t: len(str(t).strip()) == 0)
    df["code_block_present"] = df["raw_answer"].apply(lambda t: isinstance(t, str) and "```" in t)

    df["is_correct"] = df.apply(lambda r: bool(pick(r, "correct", False)), axis=1)
    df["declared_parse_err"] = df.apply(lambda r: bool(pick(r, "parse_err", False)), axis=1)
    df["parse_err_flag"] = df["json_parse_err"] | df["declared_parse_err"] | df.get("load_parse_err", False)
    df["code_err_msg_norm"] = df.get("code_err_msg", "").apply(normalize_ok)
    df["code_gen_err_norm"] = df.get("code_gen_err", "").apply(normalize_ok)
    df["code_err_flag"] = (
        df.get("code_parse_err", False)
        | df["code_gen_err_norm"].astype(str).str.strip().astype(bool)
        | df["code_err_msg_norm"].astype(str).str.strip().astype(bool)
    )
    df["issue"] = df.apply(classify_issue, axis=1)
    return df



In [12]:
df_raw = load_jsonl(RUN_PATHS, limit=MAX_ROWS)
print(f"Loaded {len(df_raw):,} rows from {len(RUN_PATHS)} file(s)")

# Derive flags for correctness + parse/code issues
with pd.option_context("mode.copy_on_write", True):
    df = derive_flags(df_raw)

df.head(3)


Loaded 15,366 rows from 39 file(s)


,request_id,unique_tag,index_in_kind,model,seed,exp_id,digit,kind,question,answer,...,json_parse_err,missing_output,code_block_present,is_correct,declared_parse_err,parse_err_flag,code_err_msg_norm,code_gen_err_norm,code_err_flag,issue
0,04a9bf60c857bac76c8dc78913787c8b258f03df,ff600ee61be5d8cd0beaf063742bdb06c6da9448,1.0,anthropic/claude-haiku-4.5,0,run_20251209_164118,0,gsm8k,Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per...,18,...,False,False,False,True,True,True,,,True,parse_error
1,fd83e8de226e91fde47d36ab1ee1b76d42f431dd,20065e9c3607252aead727be3abee819e3933adf,2.0,anthropic/claude-haiku-4.5,0,run_20251209_164118,0,gsm8k,A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?,3,...,False,False,False,True,True,True,,,True,parse_error
2,65ef33af12c9392e5fa6b319ce2be23d19f0a3f0,a3e3b525c71c2b750648586b1d3f1d8a8eca5ad3,3.0,anthropic/claude-haiku-4.5,0,run_20251209_164118,0,gsm8k,"Josh decides to try flipping a house. He buys a house for $80,000 and then puts in $50,000 in repairs. This increased the value of the house by 150%. How much profit did he make?",70000,...,False,False,False,True,True,True,,,True,parse_error


In [13]:
# Run-level stats and blanks (blanks ≈ crash/empty output)
run_stats = (
    df.groupby(["run_id", "run_model"])
    .agg(
        total=("request_id", "count"),
        blanks=("missing_output", "sum"),
        parse_err=("parse_err_flag", "sum"),
        code_err=("code_err_flag", "sum"),
    )
    .assign(blank_rate=lambda t: t["blanks"] / t["total"])
    .reset_index()
    .sort_values("blank_rate", ascending=False)
)

print("Run-level counts (including blanks; blanks imply crash/empty output)")
display(run_stats)

blanks = df[df["missing_output"]].copy()
print(f"\nBlank/missing outputs: {len(blanks)} rows")
if not blanks.empty:
    display(
        blanks[
            [
                "run_id",
                "run_model",
                "index_in_kind",
                "kind",
                "task_type",
                "answer",
                "raw_answer",
                "sim_err_msg",
                "code_err_msg",
                "code_gen_err",
            ]
        ]
    )



Run-level counts (including blanks; blanks imply crash/empty output)


,run_id,run_model,total,blanks,parse_err,code_err,blank_rate
0,run_20251205_194501,mistralai/ministral-14b-2512,0,32,100,116,inf
1,run_20251205_200043,mistralai/ministral-14b-2512,0,27,103,115,inf
2,run_20251205_201739,mistralai/ministral-14b-2512,0,31,99,116,inf
3,run_20251205_203838,mistralai/ministral-14b-2512,0,21,112,117,inf
4,run_20251205_205327,mistralai/ministral-14b-2512,0,26,93,108,inf
5,run_20251205_212912,meta-llama/llama-3-70b-instruct,0,14,117,360,inf
25,run_20251209_091023,allenai/olmo-2-0325-32b-instruct,366,366,366,366,1.000000
24,run_20251209_084725,allenai/olmo-2-0325-32b-instruct,366,366,366,366,1.000000
14,run_20251208_183719,deepseek/deepseek-r1,366,145,267,366,0.396175
15,run_20251208_201806,deepseek/deepseek-r1,366,139,287,366,0.379781



Blank/missing outputs: 3239 rows


,run_id,run_model,index_in_kind,kind,task_type,answer,raw_answer,sim_err_msg,code_err_msg,code_gen_err
10,run_20251209_164118,anthropic/claude-haiku-4.5,11.0,gsm8k,code,366,,ok,"type_check_failed,timeout","type_check_failed,timeout"
20,run_20251209_164118,anthropic/claude-haiku-4.5,21.0,gsm8k,code,15,,ok,"type_check_failed,timeout","type_check_failed,timeout"
23,run_20251209_164118,anthropic/claude-haiku-4.5,24.0,gsm8k,code,8,,ok,"type_check_failed,timeout","type_check_failed,timeout"
48,run_20251209_164118,anthropic/claude-haiku-4.5,24.0,add,code,10003392290263126,,ok,"type_check_failed,timeout","type_check_failed,timeout"
49,run_20251209_164118,anthropic/claude-haiku-4.5,1.0,sub,code,10,,ok,"type_check_failed,timeout","type_check_failed,timeout"
...,...,...,...,...,...,...,...,...,...,...
15336,run_20251209_044045,qwen/qwen-2.5-coder-32b-instruct,21.0,gcp,code,,,ok,"type_check_failed,timeout","type_check_failed,timeout"
15339,run_20251209_044045,qwen/qwen-2.5-coder-32b-instruct,24.0,gcp,code,,,ok,"type_check_failed,import * only allowed at module level (<string>, line 3)","type_check_failed,import * only allowed at module level (<string>, line 3)"
15348,run_20251209_044045,qwen/qwen-2.5-coder-32b-instruct,8.0,edp,code,,,ok,"type_check_failed,timeout","type_check_failed,timeout"
15352,run_20251209_044045,qwen/qwen-2.5-coder-32b-instruct,12.0,edp,code,,,ok,"type_check_failed,timeout","type_check_failed,timeout"


In [14]:
summary = (
    df.groupby("task_type")
    .agg(
        total=("request_id", "count"),
        correct=("is_correct", "sum"),
        parse_err=("parse_err_flag", "sum"),
        code_err=("code_err_flag", "sum"),
        missing=("missing_output", "sum"),
    )
    .assign(
        acc=lambda t: t["correct"] / t["total"],
        parse_rate=lambda t: t["parse_err"] / t["total"],
        code_err_rate=lambda t: t["code_err"] / t["total"],
    )
)

print("Per-task summary")
display(summary.sort_index())

print("\nOverall")
display(
    pd.DataFrame(
        {
            "total": [len(df)],
            "correct": [df["is_correct"].sum()],
            "parse_err": [df["parse_err_flag"].sum()],
            "code_err": [df["code_err_flag"].sum()],
            "missing": [df["missing_output"].sum()],
            "acc": [df["is_correct"].mean()],
        }
    )
)



Per-task summary


,total,correct,parse_err,code_err,missing,acc,parse_rate,code_err_rate
task_type,,,,,,,,
code,6679,6206,6772,6818,1417,0.929181,1.013924,1.020811
nl,3745,1215,2368,4538,1602,0.324433,0.632310,1.211749
sim,2446,1045,267,63,220,0.427228,0.109158,0.025756



Overall


,total,correct,parse_err,code_err,missing,acc
0,15366,8466,9407,11419,3239,0.550957


In [15]:
errors = df[df["issue"] != "ok"].copy()

print("Issue counts")
display(errors["issue"].value_counts().rename_axis("issue").to_frame("count"))

# Top code error messages (normalized; 'ok' removed)
code_msgs = df["code_err_msg_norm"].fillna("")
code_msgs = code_msgs[code_msgs != ""]
if not code_msgs.empty:
    print("\nTop code error messages")
    display(code_msgs.value_counts().head(10).rename_axis("message").to_frame("count"))

# Type-check failures are usually recorded in code_gen_err_norm
if "type_check_failed" in df.get("code_gen_err_norm", pd.Series([], dtype=str)).str.cat(sep=" "):
    tc = df[df["code_gen_err_norm"].str.contains("type_check_failed", na=False)]["code_gen_err_norm"]
    print("\nType-check related messages")
    display(tc.value_counts().head(10).rename_axis("message").to_frame("count"))

# Top combined sim/code error messages (normalized)
msg = errors.get("sim_err_msg", "").fillna("")
msg = msg.where(msg != "", df.get("code_err_msg_norm", "").fillna(""))
msg = msg.where(msg != "", df.get("code_gen_err_norm", "").fillna(""))
msg = msg[msg != ""]

if not msg.empty:
    print("\nTop error messages (any)")
    display(msg.value_counts().head(10).rename_axis("message").to_frame("count"))
else:
    print("No error messages recorded in the loaded runs.")



Issue counts


,count
issue,
parse_error,9407
code_error,2338
wrong_answer,1292
missing_output,95



Top code error messages


,count
message,
no_code,3808
No code recieved from simulation parse,793
"type_check_failed,timeout",512
"type_check_failed,invalid syntax (<string>, line 3)",155
"type_check_failed,unexpected character after line continuation character (<string>, line 1)",118
"type_check_failed,invalid syntax (<string>, line 5)",54
"type_check_failed,invalid decimal literal (<string>, line 5)",40
"type_check_failed,invalid syntax (<string>, line 6)",37
"type_check_failed,invalid character '‑' (U+2011) (<string>, line 4)",33



Type-check related messages


,count
message,
"type_check_failed,timeout",512
"type_check_failed,invalid syntax (<string>, line 3)",155
"type_check_failed,unexpected character after line continuation character (<string>, line 1)",118
"type_check_failed,invalid syntax (<string>, line 5)",54
"type_check_failed,invalid decimal literal (<string>, line 5)",40
"type_check_failed,invalid syntax (<string>, line 6)",37
"type_check_failed,invalid character '‑' (U+2011) (<string>, line 4)",33
"type_check_failed,import * only allowed at module level (<string>, line 3)",30
"type_check_failed,list indices must be integers or slices, not tuple",30



Top error messages (any)


count
message                                                                                                                                                                                                        
ok                                                                                                                                                                                                         9629
Invalid json output: \nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE                                                                           1908
Failed to parse CodeReasoning from completion null. Got: 1 validation error for CodeReasoning\n  Input should be a valid dictionary or instance of CodeReasoning [type=model_type, input_value=None, ...     46
Failed to parse CodeReasoning from completion 8993. Got: 1 validation error for CodeReasoning\n  Input should be a valid dictionary or instance of CodeReasoning [type=model_type, input_value=8993, ...      2
Invalid json output: Here is the well-formatted JSON output for the Knapsack Problem solution:\n\n```json\n{\n  "Feasible": "True",\n  "TotalValue": "21",\n  "SelectedItemIds": "[0, 2, 3, 5]",\n  "...      1
Invalid json output: ```json\n{\n  "Feasible": "True",\n  "TotalValue": "12",\n  "SelectedItemIds": "[0, 1, 3, 4]",\n  "code": "\n```python\nfrom typing import Tuple, List\nimport pulp\n\ndef solut...      1
Invalid json output: Here is the well-formatted JSON output for the Knapsack Problem solution:\n\n```json\n{\n  "Feasible": "True",\n  "TotalValue": "13",\n  "SelectedItemIds": "[0, 1, 3]",\n  "cod...      1
Invalid json output: Here is the well-formatted JSON output for the Knapsack Problem with the given items and capacity of 25:\n\n```json\n{\n  "Feasible": "True",\n  "TotalValue": "5",\n  "Selected...      1
Invalid json output: Here is the well-formatted JSON output for the Knapsack Problem solution:\n\n```json\n{\n  "Feasible": "True",\n  "TotalValue": "8",\n  "SelectedItemIds": "[1, 2, 3]",\n  "code...      1
Invalid json output: Here is the well-formatted JSON response for the Knapsack Problem with the given items and capacity of 25:\n\n```json\n{\n  "Feasible": "True",\n  "TotalValue": "13",\n  "Selec...      1

In [16]:
def render_case(row: pd.Series) -> None:
    prompt = row.get("sim_question") or row.get("code_question") or row.get("nl_question") or "<no prompt>"
    answer = row.get("raw_answer", "")
    sim_reasoning = row.get("sim_reasoning", "")
    err_msg = row.get("sim_err_msg") or row.get("code_err_msg") or row.get("code_gen_err") or ""
    parts = [
        f"### {row.get('kind', 'unknown')} · task={row.get('task_type')} · issue={row.get('issue')} · correct={row.get('is_correct')}",
        f"- model={row.get('model')} · seed={row.get('seed')} · run_id={row.get('run_id')} · idx={row.get('index_in_kind')}",
        f"- parse_err={row.get('parse_err_flag')} · code_err={row.get('code_err_flag')} · missing={row.get('missing_output')} · code_block={row.get('code_block_present')}",
        f"- expected answer: {row.get('answer')} (digit={row.get('digit')})",
        f"#### Prompt\n{prompt}",
        f"""#### Model output\n```
{answer}
```""",
    ]
    if sim_reasoning:
        parts.append(f"#### Simulated reasoning\n{sim_reasoning}")
    if err_msg:
        parts.append(f"#### Error message\n{err_msg}")
    display(Markdown("\n\n".join(parts)))


def view_samples(df: pd.DataFrame, issue: str = "parse_error", n: int = 3) -> None:
    subset = df if issue == "all" else df[df["issue"] == issue]
    for _, row in subset.head(n).iterrows():
        render_case(row)


# Example: view first few parse errors, then wrong answers
view_samples(df, issue="parse_error", n=2)
view_samples(df, issue="wrong_answer", n=2)



### gsm8k · task=code · issue=parse_error · correct=True

- model=anthropic/claude-haiku-4.5 · seed=0 · run_id=run_20251209_164118 · idx=1.0

- parse_err=True · code_err=True · missing=False · code_block=False

- expected answer: 18 (digit=0)

#### Prompt
Description: You are going to be given a set of math problem.Question: Solve the following math problems: 
 Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?FOLLOW THE FORMAT CAREFULLY. Here are the format instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Answer": {"default": "", "description": "The answer to the math problem. Type: int. Example: 1 ", "title": "Answer", "type": "string"}, "code": {"default": "", "description": "The code block that specifies a function 'solution()' that defines all variables, imports and IMPLEMENTS the actual code to solve the problem that can be executed. You may use packages Pulp, Numpy, Pandas, Torch, Scipy. Begin and end code with ```python```.Do not \"\"\"python For example an INCORRECT way to solve the problem (Don't copy method, but only formatting) but is formatted correctly:       \n\n```python\ndef solution():\n    import numpy as np\n    variable = [0,1,2,3]\n    out = np.sum(variable) \n    return int(out)\n```Here are the required types: def solution() -> int", "title": "Code", "type": "string"}, "simulation": {"default": "", "description": "The attempt at simulating the code in natural language reasoning to give the final answer.", "title": "Simulation", "type": "string"}}}
```

#### Model output
```
18
```

#### Error message
ok

### gsm8k · task=code · issue=parse_error · correct=True

- model=anthropic/claude-haiku-4.5 · seed=0 · run_id=run_20251209_164118 · idx=2.0

- parse_err=True · code_err=True · missing=False · code_block=False

- expected answer: 3 (digit=0)

#### Prompt
Description: You are going to be given a set of math problem.Question: Solve the following math problems: 
 A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?FOLLOW THE FORMAT CAREFULLY. Here are the format instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Answer": {"default": "", "description": "The answer to the math problem. Type: int. Example: 1 ", "title": "Answer", "type": "string"}, "code": {"default": "", "description": "The code block that specifies a function 'solution()' that defines all variables, imports and IMPLEMENTS the actual code to solve the problem that can be executed. You may use packages Pulp, Numpy, Pandas, Torch, Scipy. Begin and end code with ```python```.Do not \"\"\"python For example an INCORRECT way to solve the problem (Don't copy method, but only formatting) but is formatted correctly:       \n\n```python\ndef solution():\n    import numpy as np\n    variable = [0,1,2,3]\n    out = np.sum(variable) \n    return int(out)\n```Here are the required types: def solution() -> int", "title": "Code", "type": "string"}, "simulation": {"default": "", "description": "The attempt at simulating the code in natural language reasoning to give the final answer.", "title": "Simulation", "type": "string"}}}
```

#### Model output
```
3
```

#### Error message
ok

### add · task=sim · issue=wrong_answer · correct=False

- model=deepseek/deepseek-r1 · seed=4 · run_id=run_20251209_010952 · idx=19.0

- parse_err=False · code_err=False · missing=False · code_block=True

- expected answer: 16273255316448950 (digit=16)

#### Prompt
Description: You are going to be given a set of algorithmic problem.Question: Solve the following algorithmic problem: 
 Compute: 7499055461875011 + 8774199854573939FOLLOW THE FORMAT CAREFULLY. Here are the format instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Answer": {"default": "", "description": "The answer to the algorithmic problem. Type: int. Example: 1 ", "title": "Answer", "type": "string"}, "code": {"default": "", "description": "The code block that specifies a function 'solution()' that defines all variables, imports and IMPLEMENTS the actual code to solve the problem that can be executed. You may use packages Pulp, Numpy, Pandas, Torch, Scipy. Begin and end code with ```python```.Do not \"\"\"python For example an INCORRECT way to solve the problem (Don't copy method, but only formatting) but is formatted correctly:       \n\n```python\ndef solution():\n    import numpy as np\n    variable = [0,1,2,3]\n    out = np.sum(variable) \n    return int(out)\n```Here are the required types: def solution() -> int", "title": "Code", "type": "string"}, "simulation": {"default": "", "description": "The attempt at simulating the code in natural language reasoning to give the final answer.", "title": "Simulation", "type": "string"}}}
```

#### Model output
```
```json
{
  "Answer": "16273256156448950",
  "code": "```python\ndef solution() -> int:\n    num1 = 7499055461875011\n    num2 = 8774199854573939\n    total = num1 + num2\n    return total\n```",
  "simulation": "The problem requires adding two large integers: 7499055461875011 and 8774199854573939. Python's integer arithmetic handles arbitrary precision, so direct addition is sufficient. Breaking down the calculation:\n1. Assign the first number: num1 = 7499055461875011\n2. Assign the second number: num2 = 8774199854573939\n3. Compute the sum: total = num1 + num2\n   - Units digit: 1 + 9 = 10 → write 0, carry 1\n   - Tens digit: 1 (carry) + 1 + 3 = 5 → write 5, carry 0\n   - Continue digit-wise addition\n   - Final result: 16273256156448950\n4. Return the integer result: 16273256156448950"
}
```
```

#### Error message
ok

### add · task=sim · issue=wrong_answer · correct=False

- model=deepseek/deepseek-r1 · seed=4 · run_id=run_20251209_010952 · idx=22.0

- parse_err=False · code_err=False · missing=False · code_block=True

- expected answer: 17893535319013524 (digit=16)

#### Prompt
Description: You are going to be given a set of algorithmic problem.Question: Solve the following algorithmic problem: 
 Compute: 9119335464439583 + 8774199854573939FOLLOW THE FORMAT CAREFULLY. Here are the format instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Answer": {"default": "", "description": "The answer to the algorithmic problem. Type: int. Example: 1 ", "title": "Answer", "type": "string"}, "code": {"default": "", "description": "The code block that specifies a function 'solution()' that defines all variables, imports and IMPLEMENTS the actual code to solve the problem that can be executed. You may use packages Pulp, Numpy, Pandas, Torch, Scipy. Begin and end code with ```python```.Do not \"\"\"python For example an INCORRECT way to solve the problem (Don't copy method, but only formatting) but is formatted correctly:       \n\n```python\ndef solution():\n    import numpy as np\n    variable = [0,1,2,3]\n    out = np.sum(variable) \n    return int(out)\n```Here are the required types: def solution() -> int", "title": "Code", "type": "string"}, "simulation": {"default": "", "description": "The attempt at simulating the code in natural language reasoning to give the final answer.", "title": "Simulation", "type": "string"}}}
```

#### Model output
```
```json
{
  "Answer": "17893535319013522",
  "code": "```python\ndef solution():\n    num1 = 9119335464439583\n    num2 = 8774199854573939\n    total = num1 + num2\n    return total\n```",
  "simulation": "The problem requires adding two large integers: 9119335464439583 and 8774199854573939. Python handles arbitrary-precision integers, so we can directly compute their sum. The calculation proceeds as follows:\n1. Assign the first number to variable 'num1'\n2. Assign the second number to variable 'num2'\n3. Compute the sum: num1 + num2\n4. Return the result\nVerification: Adding the numbers manually from right to left with carry-over confirms the result. The final sum is 17893535319013522."
}
```
```

#### Error message
ok

In [17]:
# Convenience filters
parse_issues = df[df["issue"] == "parse_error"][
    ["kind", "task_type", "issue", "parse_err_flag", "json_parse_err", "declared_parse_err", "sim_err_msg", "code_err_msg", "raw_answer"]
]
code_issues = df[df["issue"] == "code_error"][
    ["kind", "task_type", "issue", "code_err_flag", "code_err_msg", "code_gen_err", "raw_answer"]
]
wrong_answers = df[df["issue"] == "wrong_answer"][
    ["kind", "task_type", "answer", "raw_answer", "sim_reasoning", "code_block_present"]
]

print("Parse-related issues (sample)")
display(parse_issues.head(5))

print("\nCode-related issues (sample)")
display(code_issues.head(5))

print("\nWrong answers (sample)")
display(wrong_answers.head(5))



Parse-related issues (sample)


,kind,task_type,issue,parse_err_flag,json_parse_err,declared_parse_err,sim_err_msg,code_err_msg,raw_answer
0,gsm8k,code,parse_error,True,False,True,ok,"ok,ok",18
1,gsm8k,code,parse_error,True,False,True,ok,"ok,ok",3
2,gsm8k,code,parse_error,True,False,True,ok,"ok,ok",70000
3,gsm8k,code,parse_error,True,False,True,ok,"ok,ok",540
4,gsm8k,code,parse_error,True,False,True,ok,"ok,ok",20



Code-related issues (sample)


,kind,task_type,issue,code_err_flag,code_err_msg,code_gen_err,raw_answer
116,lcs,nl,code_error,True,no_code,no_code,"```json\n{\n ""Answer"": ""4"",\n ""simulation"": ""To find the Longest Common Subsequence (LCS) between S = 'lcovqdyfqmlpxapb' and T = 'bcclfxzvjitgtbsv', I need to identify the longest sequence of ch..."
120,lcs,nl,code_error,True,no_code,no_code,"```json\n{\n ""Answer"": ""3"",\n ""simulation"": ""To find the Longest Common Subsequence (LCS) between S = 'plnlarrtztkotazh' and T = 'bcclfxzvjitgtbsv', I need to identify the longest sequence of ch..."
183,ilp_assign,nl,code_error,True,no_code,no_code,"```json\n{\n ""simulation"": ""The assignment problem requires finding a one-to-one matching between 8 workers and 8 tasks that minimizes total cost. I'll use a greedy approach with optimization:\n\..."
189,ilp_assign,nl,code_error,True,no_code,no_code,"```json\n{\n ""Answer"": ""118"",\n ""simulation"": ""To solve this 16×16 assignment problem, I need to find the optimal one-to-one matching between workers and tasks that minimizes total cost. This is..."
190,ilp_assign,nl,code_error,True,no_code,no_code,"```json\n{\n ""Answer"": ""118"",\n ""simulation"": ""To solve this 16×16 assignment problem, I need to find the optimal assignment of workers to tasks that minimizes total cost. This is a classic Hung..."



Wrong answers (sample)


,kind,task_type,answer,raw_answer,sim_reasoning,code_block_present
2605,add,sim,16273255316448950,"```json\n{\n ""Answer"": ""16273256156448950"",\n ""code"": ""```python\ndef solution() -> int:\n num1 = 7499055461875011\n num2 = 8774199854573939\n total = num1 + num2\n return total\n```...",,True
2608,add,sim,17893535319013524,"```json\n{\n ""Answer"": ""17893535319013522"",\n ""code"": ""```python\ndef solution():\n num1 = 9119335464439583\n num2 = 8774199854573939\n total = num1 + num2\n return total\n```"",\n ""...",,True
2623,sub,sim,3785463,"```json\n{\n ""Answer"": ""-3785463"",\n ""code"": ""```python\ndef solution() -> int:\n num1 = 13804733\n num2 = 17590196\n result = num1 - num2\n return result\n```"",\n ""simulation"": ""To...",,True
3550,ilp_assign,sim,64,"\n```json\n{\n ""Answer"": ""67"",\n ""code"": ""```python\ndef solution():\n from scipy.optimize import linear_sum_assignment\n C = [[18, 10, 11, 15, 5, 20, 28, 29, 31, 12, 23, 13, 28, 5, 18, 14...",,True
3551,ilp_prod,sim,0,"\n```json\n{\n ""Answer"": ""40"",\n ""code"": ""```python\ndef solution():\n import pulp\n\n # Define the problem data\n profit = [6, 20]\n consumption = [[29, 27], [26, 4]]\n capacity ...",,True
